## Processing ETL des sets de données Olist

#### 1. Nettoyage: Opérations globales à tous les fichiers source

Dans cette partie je gère les nettoyages qui peuvent potentiellement s'appliquer à toutes les tables, sans faire de traitement spécifique pour telle ou telle table

In [2]:
# Import de dépendances et chargement des datasets en mémoire vive

import os
from pathlib import Path
import pandas as pd
from scripts.utils import clean_data

data = {}

rt = Path('./data')
file_paths = os.listdir(rt)
csv_file_paths = [f for f in file_paths if f.endswith('.csv')]
for csv_path in csv_file_paths:
    print(f'Loading {csv_path}...')
    df = pd.read_csv(rt.joinpath(csv_path))
    df = clean_data(df) # Utilisation d'un heler générique de nettoyage
    source = csv_path.replace('olist_', '').replace('.csv', '').replace('_dataset', '')
    data[source] = df

Loading olist_orders_dataset.csv...


/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely

Loading olist_products_dataset.csv...
Loading olist_order_items_dataset.csv...


/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],


Loading product_category_name_translation.csv...
Loading olist_sellers_dataset.csv...
Loading olist_geolocation_dataset.csv...
Loading olist_order_reviews_dataset.csv...


/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/scripts/utils.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],


Loading olist_order_payments_dataset.csv...
Loading olist_customers_dataset.csv...


#### 2. Nettoyage: Cas spécifiques

Dans cette partie du notebook j'applique des netoyages plus spécifiques, sur des tables en particulier:
- customers: suppression de doublons et d"une colonne non utilisée => EDIT: Pas une bonne idée, je commente
- geolocation et customers: normalisation des noms de ville pour cohérence des éventuels PKI qui utiliseraient les noms de ville

In [ ]:
# Nettoyage des doublons de données table customers
# data['customers'].drop_duplicates(subset=['customer_unique_id'], keep='first')
# data['customers'] = data['customers'].drop(columns=['customer_unique_id'])

In [3]:
from scripts.utils import norm

# Normalise les noms de villes
data['geolocation']['geolocation_city'] = data['geolocation']['geolocation_city'].apply(norm)
data['customers']['customer_city'] = data['customers']['customer_city'].apply(norm)

#### 3. Transformations

La transformation principale que j'ai imaginée sur ce cas d'école a été d'ajouter les traductions de noms de catégories directement dans la table produits.

Le but étant de simplifier les requêtes devant filtrer ou grouper par catégorie, cela évite d'avoir à faire des jointures ultérieures pour récupérer les traductions.

Au départ j'ai même considéré faire cela pour cmplètement supprimer la table de traductions, mais j'ai décidé de la garder, dans le scénario où un utilisateur brésilien aurait besoin de faire des requêtes sur la base de données: dans ce cas les jointures resteront faisables.

In [4]:
# Transformation de la table des produits

# Merge des dataframes pour ajouter la traduction anglaise et éviter les jointures
data['products'] = data['products'].merge(
    data['product_category_name_translation'],
    how='left', # Type de jointure : left pour conserver tous les produits même sans traduction
    left_on='product_category_name', # Clé de jointure dans la table des produits
    right_on='product_category_name' # Clé de jointure dans la table de traduction
)

# Renommer la nouvelle colonne pour éviter les conflits
data['products'].rename(columns={'product_category_name_english': 'product_category_name_en'},
                inplace=True)

# Résultat final
print(data['products'][['product_category_name', 'product_category_name_en']].head())

   product_category_name product_category_name_en
0             perfumaria                perfumery
1                  artes                      art
2          esporte_lazer           sports_leisure
3                  bebes                     baby
4  utilidades_domesticas               housewares


#### 4. Chargement en base de données


Pour le chargement j'ai choisi d'utiliser sqlite3 pour gérer la partie création de tables en manuel avec des scripts SQL; ça permet d'avoir une création de tables qui inclue directement toutes les contraintes nécessaires, et c'est plus simple que d'utiliser la syntaxe de sqlalchemy, qui ajoute une couche d'abstraction supplémentaire à SQL.

In [5]:
import sqlite3

DB_PATH = "./olist.db"
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()

    # Ouverture et exécution du script SQL de création de schéma de base de données
    with open('./scripts/schema.sql', 'r') as f:
        schema_sql = f.read()
        cur.executescript(schema_sql)

In [3]:
# Chargement des données nettoyées/transformées dans la base de données SQLite
for table_name, df in data.items():
    df.to_sql(table_name, con=conn, if_exists='replace', index=False)

NameError: name 'data' is not defined

### 5. Développement d'indicateurs de perfomances

In [12]:
# Setup du système de benchmark
%load_ext autoreload
%autoreload 2

from scripts.utils import QueryBenchmark
import importlib
import scripts
importlib.reload(scripts.utils) # Recharger le module pour prendre en compte les modifications récentes
benchmark = QueryBenchmark()
format_stats = lambda rows, elapsed: f"Fetched {rows} rows in {elapsed:.3f}s"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 💰 Ventes

In [ ]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/sales/daily.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("sales", "daily", "raw", rows_processed, execution_time)
    
    print(format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Fetched 633 rows in 0.390s
Résultat (max. 20):
{'jour': '2016-09-04', 'ca_jour': 136.23}
{'jour': '2016-09-05', 'ca_jour': 75.06}
{'jour': '2016-09-13', 'ca_jour': 40.95}
{'jour': '2016-10-02', 'ca_jour': 109.34}
{'jour': '2016-10-03', 'ca_jour': 595.14}
{'jour': '2016-10-04', 'ca_jour': 11914.65}
{'jour': '2016-10-05', 'ca_jour': 10212.22}
{'jour': '2016-10-06', 'ca_jour': 9398.61}
{'jour': '2016-10-07', 'ca_jour': 8255.19}
{'jour': '2016-10-08', 'ca_jour': 9716.13}
{'jour': '2016-10-09', 'ca_jour': 4152.11}
{'jour': '2016-10-10', 'ca_jour': 4675.1}
{'jour': '2016-10-22', 'ca_jour': 61.99}
{'jour': '2016-12-23', 'ca_jour': 19.62}
{'jour': '2017-01-05', 'ca_jour': 707.27}
{'jour': '2017-01-06', 'ca_jour': 988.28}
{'jour': '2017-01-07', 'ca_jour': 1447.99}
{'jour': '2017-01-08', 'ca_jour': 809.63}
{'jour': '2017-01-09', 'ca_jour': 774.51}
{'jour': '2017-01-10', 'ca_jour': 1571.17}


In [ ]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/sales/monthly.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("sales", "monthly", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 25 rows in 0.416s
Résultat (max. 20):
{'mois': '2016-09', 'ca_mois': 252.24}
{'mois': '2016-10', 'ca_mois': 59090.48}
{'mois': '2016-12', 'ca_mois': 19.62}
{'mois': '2017-01', 'ca_mois': 138488.04}
{'mois': '2017-02', 'ca_mois': 291908.01}
{'mois': '2017-03', 'ca_mois': 449863.6}
{'mois': '2017-04', 'ca_mois': 417788.03}
{'mois': '2017-05', 'ca_mois': 592918.82}
{'mois': '2017-06', 'ca_mois': 511276.38}
{'mois': '2017-07', 'ca_mois': 592382.92}
{'mois': '2017-08', 'ca_mois': 674396.32}
{'mois': '2017-09', 'ca_mois': 727762.45}
{'mois': '2017-10', 'ca_mois': 779677.88}
{'mois': '2017-11', 'ca_mois': 1194882.8}
{'mois': '2017-12', 'ca_mois': 878401.48}
{'mois': '2018-01', 'ca_mois': 1115004.18}
{'mois': '2018-02', 'ca_mois': 992463.34}
{'mois': '2018-03', 'ca_mois': 1159652.12}
{'mois': '2018-04', 'ca_mois': 1160785.48}
{'mois': '2018-05', 'ca_mois': 1153982.15}


In [ ]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/sales/yearly.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("sales", "yearly", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 3 rows in 0.390s
Résultat (max. 20):
{'annee': '2016', 'ca_annee': 59362.34}
{'annee': '2017', 'ca_annee': 7249746.73}
{'annee': '2018', 'ca_annee': 8699763.05}


In [ ]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/sales/previous_year_comparison.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("sales", "previous_year_comparison", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 633 rows in 0.779s
Résultat (max. 20):
{'jour': '2016-09-04', 'ca_courant': 136.23, 'ca_n_1': 28719.24, 'variation_abs': -28583.010000000002, 'variation_pct': -99.53}
{'jour': '2016-09-05', 'ca_courant': 75.06, 'ca_n_1': 24954.68, 'variation_abs': -24879.62, 'variation_pct': -99.7}
{'jour': '2016-09-13', 'ca_courant': 40.95, 'ca_n_1': 32054.37, 'variation_abs': -32013.42, 'variation_pct': -99.87}
{'jour': '2016-10-02', 'ca_courant': 109.34, 'ca_n_1': 25565.11, 'variation_abs': -25455.77, 'variation_pct': -99.57}
{'jour': '2016-10-03', 'ca_courant': 595.14, 'ca_n_1': 30389.12, 'variation_abs': -29793.98, 'variation_pct': -98.04}
{'jour': '2016-10-04', 'ca_courant': 11914.65, 'ca_n_1': 24787.58, 'variation_abs': -12872.930000000002, 'variation_pct': -51.93}
{'jour': '2016-10-05', 'ca_courant': 10212.22, 'ca_n_1': 29129.83, 'variation_abs': -18917.61, 'variation_pct': -64.94}
{'jour': '2016-10-06', 'ca_courant': 9398.61, 'ca_n_1': 21890.58, 'variation_abs': -12491.97

In [17]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/sales/top10.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("sales", "top10", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 10 rows in 0.677s
Résultat (max. 20):
{'product_id': '5769ef0a239114ac3a854af00df129e4', 'product_category_name_en': 'fixed_telephony', 'product_name_length': 39.0, 'ca_produit': 109312.64, 'rang': 1}
{'product_id': 'bb50f2e236e5eea0100680137654686c', 'product_category_name_en': 'health_beauty', 'product_name_length': 60.0, 'ca_produit': 81887.42, 'rang': 2}
{'product_id': '422879e10f46682990de24d770e7f83d', 'product_category_name_en': 'garden_tools', 'product_name_length': 56.0, 'ca_produit': 79512.22, 'rang': 3}
{'product_id': 'd1c427060a0f73f6b889a5c7c61f2ac4', 'product_category_name_en': 'computers_accessories', 'product_name_length': 59.0, 'ca_produit': 70557.9, 'rang': 4}
{'product_id': '6cdd53843498f92890544667809f1595', 'product_category_name_en': 'health_beauty', 'product_name_length': 51.0, 'ca_produit': 64825.67, 'rang': 5}
{'product_id': 'd5991653e037ccb7af6ed7d94246b249', 'product_category_name_en': 'computers_accessories', 'product_name_length': 42.0

#### 👥 Clients

In [18]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/customers/new_vs_recurring_customers.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("customers", "new_vs_recurring_customers", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 1 rows in 0.060s
Résultat (max. 20):
{'new_customers': 99441, 'returning_customers': 0}


In [19]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/customers/average_cart.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("customers", "average_cart", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 1 rows in 0.052s
Résultat (max. 20):
{'avg_basket_value': 160.57763809214927}


In [20]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/customers/conversion_rate.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("customers", "conversion_rate", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 1 rows in 0.013s
Résultat (max. 20):
{'conversion_rate': 0.9702034372140264}


In [21]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/customers/rfmbbq_analysis.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("customers", "rfmbbq_analysis", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 98666 rows in 0.509s
Résultat (max. 20):
{'customer_id': '1617b1357756262bfa56ab541c47bc16', 'recency_days': 3059, 'frequency': 1, 'monetary': 13664.08}
{'customer_id': 'ec5b2ba62e574342386871631fafd3fc', 'recency_days': 2770, 'frequency': 1, 'monetary': 7274.88}
{'customer_id': 'c6e2731c5b391845f6800c97401a43a9', 'recency_days': 3288, 'frequency': 1, 'monetary': 6929.31}
{'customer_id': 'f48d464a0baaea338cb25f816991ab1f', 'recency_days': 2760, 'frequency': 1, 'monetary': 6922.21}
{'customer_id': '3fd6777bbce08a352fddd04e4a7cc8f6', 'recency_days': 3187, 'frequency': 1, 'monetary': 6726.66}
{'customer_id': '05455dfa7cd02f13d132aa7a6a9729c6', 'recency_days': 3003, 'frequency': 1, 'monetary': 6081.54}
{'customer_id': 'df55c14d1476a9a3467f131269c2477f', 'recency_days': 3240, 'frequency': 1, 'monetary': 4950.34}
{'customer_id': 'e0a2412720e9ea4f26c1ac985f6a7358', 'recency_days': 2773, 'frequency': 1, 'monetary': 4809.44}
{'customer_id': '24bbf5fd2f2e1b359ee7de94defc4a1

### 📊 Cohortes

In [22]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/cohorts/first_month_retention.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("cohorts", "first_month_retention", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 634 rows in 0.397s
Résultat (max. 20):
{'cohort_date': '2016-09-04', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-05', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-13', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-09-15', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-10-02', 'cohort_month': 0, 'active_customers': 1}
{'cohort_date': '2016-10-03', 'cohort_month': 0, 'active_customers': 8}
{'cohort_date': '2016-10-04', 'cohort_month': 0, 'active_customers': 63}
{'cohort_date': '2016-10-05', 'cohort_month': 0, 'active_customers': 47}
{'cohort_date': '2016-10-06', 'cohort_month': 0, 'active_customers': 51}
{'cohort_date': '2016-10-07', 'cohort_month': 0, 'active_customers': 46}
{'cohort_date': '2016-10-08', 'cohort_month': 0, 'active_customers': 42}
{'cohort_date': '2016-10-09', 'cohort_month': 0, 'active_customers': 26}
{'cohort_date': '2016-10-10', 'cohort_month': 0, 'active_customers': 

In [23]:
from scripts.utils import get_perf_stats

with open('./scripts/pki/cohorts/lifetime_value_per_cohort.sql', 'r') as f:
    sql_script = f.read()
    result, rows_processed, execution_time = get_perf_stats(sql_script)
    benchmark.set_stats("cohorts", "lifetime_value_per_cohort", "raw", rows_processed, execution_time)
    
    print('Statistiques:', format_stats(rows_processed, execution_time))
    print('Résultat (max. 20):')
    for row in result:
        print(dict(row))


Statistiques: Fetched 616 rows in 0.757s
Résultat (max. 20):
{'cohort_date': '2016-09-04', 'customers_in_cohort': 1, 'total_revenue': 136.23, 'ltv_per_customer': 136.23}
{'cohort_date': '2016-09-05', 'customers_in_cohort': 1, 'total_revenue': 75.06, 'ltv_per_customer': 75.06}
{'cohort_date': '2016-09-15', 'customers_in_cohort': 1, 'total_revenue': 143.46, 'ltv_per_customer': 143.46}
{'cohort_date': '2016-10-02', 'customers_in_cohort': 1, 'total_revenue': 109.34, 'ltv_per_customer': 109.34}
{'cohort_date': '2016-10-03', 'customers_in_cohort': 8, 'total_revenue': 595.14, 'ltv_per_customer': 74.3925}
{'cohort_date': '2016-10-04', 'customers_in_cohort': 60, 'total_revenue': 11295.48, 'ltv_per_customer': 188.25799999999998}
{'cohort_date': '2016-10-05', 'customers_in_cohort': 42, 'total_revenue': 9645.94, 'ltv_per_customer': 229.6652380952381}
{'cohort_date': '2016-10-06', 'customers_in_cohort': 49, 'total_revenue': 9131.23, 'ltv_per_customer': 186.35163265306122}
{'cohort_date': '2016-10-0